In [1]:
#Testing with the MCMC given a model functions mira

In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
%matplotlib inline
import pickle
import seaborn as sns
import sys
import numpy as np
import scipy.optimize as op
from scipy.optimize import curve_fit

sns.set_palette("PuBuGn_d",15)
sns.set_style("whitegrid")
import emcee

NameError: name 'sns' is not defined

In [ ]:
file=open( '../../data/raw_data/20160428_1313_W1204_ref.pkl' , 'rb')
#   20160428_1313_W1204_ref.pkl
d=pickle.load(file)
file.close()

i = 8
Nsize = 2**i
dt=(1/(d['sample freq'])) #period
T1=dt*(Nsize) #full period
v=(d['speed'])
X = v*T1 #full distance
dx = dt*v #smallest amount of distance travelled
total_t = (d['scan time']) #how long it ran
    
total_s = (d['samples requested']) #number of samples 
startpt = ((total_s - Nsize)/2) #starting point


endpt = startpt + Nsize #ending point


df = 1/T1
f = df*np.arange(Nsize/2)+df/2.0
fFull = df*np.arange((Nsize/2) + 1)+df/2.0


y = (d['sig0F']) 
D = y[startpt:endpt]
D = np.flipud(D)

a = d['delay0F']/v
t = a[startpt:endpt]

In [ ]:
#model function convoluted with RC circuit and AC coupling function
def psi1(znot,q,p,T):
    Ns = 256
    z = t
    dz = z[2]-z[1]
    f = np.linspace(0,0.5/dz, np.floor(Ns/2))
    f_full = np.linspace(0,0.5/dz, np.floor(Ns/2) +1)
    #original function
    y0=-np.cos(2*np.pi*q*(z-znot))*(np.exp(-((z-znot)**2)/(2*p**2)))/(np.sqrt(2*np.pi)*p)
    yf0=np.fft.rfft(y0)
    K_det =(1/(1+(1j*2*np.pi*f_full*T))) #kernel from detector
    K_cc =(2*np.pi*f_full*.559)/(1 + (1j*2*np.pi*.559)) #kernel from AC coupling
    K_f = np.multiply(K_det,K_cc)
    y1=np.fft.irfft(yf0*K_f*2*np.pi)
    return t,y1
#znot = 0.012, q = 8.57718120805, p = .04

In [ ]:
t,D = psi1(-.03, 10, .05, .049) #expected values
D = D*.0006

In [ ]:
pl.plot(t,D)

In [ ]:
#likelihood function
#why is it ln?
def lnlike(params, t, D):
    znot, q, p, T, a = params 
    t,Modelf = psi1(znot,q,p, T)
    Dev = -.5*(np.dot((D-Modelf*a),D-Modelf*a))
    Sig=0.0005  
    return (Dev/Sig)-np.log(1/Sig) #sigma? if sigma is large it becomes shallower

In [ ]:
nll = lambda *args: -lnlike(*args) 
result = op.minimize(nll, [-.03, 10, .05, .049, .0006], args=(t,D))
znot1,q1,p1,T1,a1 = result["x"]
#print result ["x"]

In [ ]:
#prior function
#again, why is it ln?
def lnprior(params):
    znot, q, p, T, a = params 
    if -.08 < znot < .08 and 1.0 < q < 15.0 and .01 < p < .1 and 0.0 < T < 1 and 0.0 < a < .1:
        return 0.0
    return -np.inf 
    

In [ ]:
#log probability function
def lnprob (params, t, D):
    lp = lnprior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(params, t, D)

In [ ]:
ndim, nwalkers = 5,260
pos = [result["x"] + 10**(-4)*np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers,ndim,lnprob, args=(t,D))

In [ ]:
sampler.run_mcmc(pos, 5000); #number of steps

In [ ]:
walks=(sampler.chain)
print(np.shape(walks))

for nn in range(0,ndim):
    pl.plot(walks[0,:,nn])
    pl.plot(walks[1,:,nn])
    pl.plot(walks[2,:,nn])
    titles = ['znot','q','p','T','a']
    pl.title(titles[nn])
    pl.show() #sigma = .001

In [ ]:
samples = sampler.chain[:,2500:,:].reshape([-1, ndim])
#print(np.shape(samples))

In [ ]:
import corner
fig = corner.corner(samples, labels=["$znot$", "$q$", "$p$", "$T$", "$a$"], 
                    truths=[-.03, 10, .05, .049, .0006])
fig.savefig("MCMCattempt2.png")

In [ ]:
samples[:, 2] = np.exp(samples[:, 2])
znot, q, p, T,a = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))

In [ ]:
print 'znot: ', znot
print 'q: ', q 
print 'p: ', p 
print 'T: ', T
print 'a: ', a

In [ ]:
#t,D = psi1(-.03, 10, .05, .049) expected values
#D = D*.0006, p is still off weirdly by about 1. (it's p +1... why?)